## Journal post-processing 

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
import cv2
import coco
import utils
import model as modellib
import visualize
import pandas as pd
from tesserocr import PyTessBaseAPI, PSM
import re
import datetime
import csv
import random

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")
#IMAGE_DIR = os.path.abspath('C:\\Users\\14444043\\Desktop\\video\\scrins_crop2')
#IMAGE_DIR = os.path.abspath('D:\\caltech pedestrian detection\\images100')

Using TensorFlow backend.


In [17]:
import csv
import pandas as pd
import numpy as np

CALENDAR_DIR='D:\\_CV_PROJECT\\calendar3\\'

In [18]:
with open('D:\\_CV_PROJECT\\journal3.csv',"rt") as csvfile:
    journal=csv.reader(csvfile)
    journal=list(journal)

In [19]:
data = pd.DataFrame(journal[1:])
data.columns=journal[0][1:]

Сортировка в хронологическом порядке

In [20]:
data=data[data['date']!='date']
data.sort_values(['image directory'],inplace=True)
data['date order']=0
k=0
for i in range(1,len(data.index[1:])):
    k = k if data['date'][data.index[1:][i]]==data['date'][data.index[1:][i-1]] else k+1
    data=data.set_value(data.index[1:][i],'date order',k)


In [21]:
data['person order']=0
data_new=pd.DataFrame(columns=data.columns)
for date in list(np.unique(data['date order'])):
    data_i=data[data['date order']==date].sort_values(['image directory'])
    first_flg=0
    for i in range(len(data_i.index)):
        if int(data_i['number of persons'][data_i.index[i]])>0:
            first_flg=1
            data_i=data_i.set_value(data_i.index[i],'person order',1)
        elif int(data_i['number of persons'][data_i.index[i]])==0 and first_flg==1:
            data_new=pd.concat([data_new,data_i])
            break
        elif i==len(data_i.index)-1 and first_flg==0:
            data_i=data_i.set_value(data_i.index[i],'person order',1)
            data_new=pd.concat([data_new,data_i])
        
    

In [22]:
data=data_new

In [23]:
data.head()

,image directory,date,time,number of persons,rois,date order,person order
1672,D:\_CV_PROJECT\scrins6\img_16-12-02.jpg,2018-01-03,08:44:01,0,,0,0
2309,D:\_CV_PROJECT\scrins6\img_16-12-04.jpg,2018-01-03,08:45:37,0,,0,0
1489,D:\_CV_PROJECT\scrins6\img_16-12-06.jpg,2018-01-03,08:47:07,0,,0,0
1341,D:\_CV_PROJECT\scrins6\img_16-12-08.jpg,2018-01-03,08:49:45,0,,0,0
596,D:\_CV_PROJECT\scrins6\img_16-12-10.jpg,2018-01-03,08:51:37,0,,0,0


## Calendar's creation

Календарь - файл, формируемый на основе журнала. В отличие от журнала, в нем хранятся только записи о моменте открытия: дата, время, имя скриншота и количество обнаруженных людей. То есть каждому дню соответствует одна запись.

Моментом открытия считается первый промежуток от появления первого человека в кадре до первого отсутствия людей в кадре.
Для подсчета количества людей в этот временной интервал, берется максимум по количеству обнаруженных людей на скриншотах, соответствующих ему.

Так сделано, потому что экспериментально это показало наилучший результат для устранения следующих проблемы появления людей последовательно друг за другом, появления людей кусками, необнаружения алгоритмом человека в определенной позе или под определенным углом.

In [9]:
calendar=data[data['person order']==1].groupby(['date order']).\
    apply(lambda x: x.sort_values('number of persons', ascending=False)[['date','time','image directory','number of persons']][:1])
    #[0, 0, 1, 2, 3, 0] от нуля до нуля, из них - максимум - [:1], 1й, тк отсортировали

In [10]:
calendar['image directory']=[i[:3]+i[3:] for i in calendar['image directory']]

In [11]:
calendar.to_excel(CALENDAR_DIR+'calendar.xlsx')

В папку с календарем выгружаются все скриншоты, которые стали решающими для каждого дня, с отрисованными на них масками, чтобы было легче анализировать результаты.

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 123.7  116.8  103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINI

In [ ]:
for i in range(len(calendar)):
    
    image = skimage.io.imread(calendar.iloc[i]['image directory'])

    r = model.detect([image], verbose=1)[0]
    persons=r['class_ids']==1

    img = visualize.display_instances(image, r['rois'][persons], r['masks'][:,:,persons], 
                                      r['class_ids'][persons], class_names, r['scores'][persons])
    cv2.imwrite(CALENDAR_DIR+calendar.iloc[i]['date']+'.jpg',img)
cv2.imshow('',img)
k=cv2.waitKey(0)
cv2.destroyAllWindows()